In [3]:
import torch
import numpy as np
from torchvision import datasets, transforms

# Define the data transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load the MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Calculate the number of samples for 5% of the training data
subsample_size = int(0.05 * len(train_dataset))

# Create a random subset of indices for the training data
subset_indices = torch.randperm(len(train_dataset))[:subsample_size]

# Use the subset indices to create the subsampled dataset
subsampled_train_dataset = torch.utils.data.Subset(train_dataset, subset_indices)

# Create data loaders for the subsampled training data and test data
train_loader = torch.utils.data.DataLoader(subsampled_train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [2]:
from bayesian_optimization import bayesian_optimization
from autoencoder import GeometricAutoencoder
from train import train_and_evaluate
from graph_search_space import construct_graph_search_space

n_p = 5
latent_dim = n_p*2
adjacency_matrix, signatures = construct_graph_search_space(n_p=n_p)

# Define the objective function for Bayesian optimization
def objective_function(signature):
    model = GeometricAutoencoder(signature, latent_dim=latent_dim)
    train_losses, test_loss = train_and_evaluate(model, train_loader, test_loader)
    return train_losses, test_loss

# Perform Bayesian optimization
evaluated_signatures = []
evaluated_metrics = []
loss_trajectories = []

def callback(signature, metric):
    evaluated_signatures.append(signature)
    loss_trajectories.append(metric[0])  
    evaluated_metrics.append(metric[1])  

optimal_signature, optimal_val_metric, optimal_train_metric = bayesian_optimization(objective_function, signatures, adjacency_matrix, n_iterations=10, callback=callback)

print("Optimal signature:", optimal_signature)
print("Optimal validation metric:", optimal_val_metric)



Training Loop:   0%|          | 0/10 [00:00<?, ?it/s]


TypeError: abs(): argument 'input' (position 1) must be Tensor, not int

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def visualize_optimization_trajectory(adjacency_matrix, signatures):
    # Create a graph object from the adjacency matrix
    graph = nx.from_numpy_array(adjacency_matrix)

    # Assign labels to the nodes based on the signatures
    labels = {i: str(signature) for i, signature in enumerate(signatures)}
    nx.set_node_attributes(graph, labels, 'signature')


    # Visualize the graph search space with the optimization trajectory
    pos = nx.spring_layout(graph)
    nx.draw_networkx_nodes(graph, pos, node_size=300, node_color='lightblue')
    nx.draw_networkx_edges(graph, pos, edge_color='gray', width=1.0)
    nx.draw_networkx_labels(graph, pos, labels, font_size=10)

    # Highlight the optimization trajectory
    evaluated_indices = [signatures.index(signature) for signature in evaluated_signatures]
    trajectory_edges = [(evaluated_indices[i], evaluated_indices[i+1]) for i in range(len(evaluated_indices)-1)]
    nx.draw_networkx_edges(graph, pos, edgelist=trajectory_edges, edge_color='red', width=2.0)

    plt.axis('off')
    plt.title('Graph Search Space with Optimization Trajectory')
    plt.show()

def visualize_validation_metrics(evaluated_metrics):
    # Visualize the metric evolution
    plt.figure()
    plt.plot(evaluated_metrics)
    plt.xlabel('Iteration')
    plt.ylabel('Validation Metric')
    plt.title('Metric Evolution during Optimization')
    plt.show()

def visualize_loss_trajectories(loss_trajectories):
    plt.figure()
    colors = plt.cm.viridis(np.linspace(0, 1, len(loss_trajectories)))
    for i, trajectory in enumerate(loss_trajectories):
        plt.plot(trajectory, color=colors[i], alpha=0.7)
    plt.xlabel('Epoch')
    plt.ylabel('Train Loss')
    plt.title('Loss Trajectories for Different Geometries')
    plt.colorbar(plt.cm.ScalarMappable(cmap='viridis'), label='Optimization Iteration')
    plt.show()


visualize_optimization_trajectory(adjacency_matrix, signatures)
visualize_validation_metrics(evaluated_metrics)
visualize_loss_trajectories(loss_trajectories)


## Misc Visualizations